In [122]:
from docx import *
import re

In [123]:
document = Document("Converted/just ind-eng-1-69.docx")

In [149]:
# for paragraph in document.paragraphs:
#     # first line indent (segitiga atas di ruler)
#     first = paragraph.paragraph_format.first_line_indent
#     left = paragraph.paragraph_format.left_indent
#     print(first,left,paragraph.text)

In [150]:
#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    #variabel
    lema = ""
    sublema = ""
    similar = ""
    frasa = ""
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #jika first dan left bernilai None sudah pasti dia lema (autoset first = -151765 dan left = 152400) 
    if first == None and left == None:
        first = -151765
        left = 152400
        
    #jika first dan left tidak bernilai None   
    else:
        #jika nilai first == None maka nilai first = tempFirst (nilai first indent baris sebelumnya / first indent warisan)
        if first == None:
            first = tempFirst

        #jika nilai left == None maka nilai left = tempLeft (nilai left indent baris sebelumnya / left indent warisan)
        if left == None:
            left = tempLeft
        
    #lema
    #indent lema adalah first = -151765 dan left = 152400
    if first <= -151765 and left >= 152400:
        
        #set temp first dan left indent untuk nilai first dan left indent warisan 
        tempFirst = -151765
        tempLeft = 152400
        
        #looping setiap kata di setiap barisnya
        for run in paragraph.runs:
            
            #jika kata style bold
            if run.bold:
                
                #jika kata style bold bukan spasi
                if run.text.isspace() == False :

                    #memecah string kata style bold menjadi array 
                    #(untuk mengatasi angka romawi dibelakang kata, ex: apa I,apa II, jadi diambil array ke-0)
                    split_text = run.text.split()
    
                    #jika split_text bukan angka
                    if split_text[0].isdigit() == False :
                
                        #jika lema kosong atau belum terdapat lema, lema = split_text[0]
                        if lema == "" : 
                            
                            lema = split_text[0]
                            print("lema = "+lema)

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :

                            #jika kata sebelum kata style bold terdapat . (titik) maka dianggap sebagai lema 
                            if re.search("[.]", paragraph.runs[index-1].text):
                                lema = split_text[0]
                                print("lema = "+lema)
                                
                            #jika kata sebelum kata style bold adalah and maka dianggap sebagai similar
                            elif paragraph.runs[index-1].text == "and" :
                                similar = split_text[0]
                                print("similar = "+similar)
                            
                            #jika bukan kedua kondisi diatas maka dianggap sublema
                            else :
                                sublema = split_text[0]
                                print("sublema = "+sublema)
            
            #index dijumlah disetiap untuk mengetahui posisi kata terletak pada index keberapa
            index += 1

    #sublema 
    else :
        tempFirst = -75565
        tempLeft = 76200
        
#         for run in paragraph.runs:
#             if run.bold:
#                 if run.text.isspace() == False :
# #                     print(paragraph.text)
# #                     print("======")
#                     print("sublema = "+run.text)

#     print([lema,sublema,similar])

lema = a
similar = A
lema = a
lema = a
lema = a
lema = A
lema = A
lema = A
lema = aah
lema = aala
similar = a’ala
lema = aam
lema = aau
lema = ab
lema = ab
lema = AB
lema = aba
lema = aba
lema = aba
lema = abab
lema = abad
lema = ab’ad
lema = abadi
sublema = ketidak-abadian
lema = abadiah
similar = abadiat
lema = abah
lema = abah
lema = abah
lema = abai
sublema = mengabaikan
lema = abaimana
lema = abaka
lema = abakus
lema = abal-abal
lema = abal-abal
lema = abandonemén
lema = abandonir
sublema = mengabandonir
lema = abang
lema = abang
lema = abang
lema = abang-abang
lema = abangan
lema = abangan
lema = abar
lema = abar
lema = Abasa
lema = Abassiyah
lema = abasti
lema = abat
lema = abat
lema = abaté
lema = abatisasi
lema = abatoar
lema = abau
lema = abawi
lema = abaya
lema = abc
similar = ABC
lema = abdi
lema = abdidalemisme
lema = abdikasi
lema = abdis
lema = abdomén
lema = abdominal
lema = abdu
lema = abécé
lema = abégé
lema = abelur
lema = abén
lema = aberasi
lema = abésé
lema = abet

lema = akrab
lema = kekurangakraban­
lema = Akrab
lema = akram
lema = akrédit
sublema = berakrédit
lema = akréditasi
lema = akrési
lema = akriditas
lema = akrilik
lema = akrilonitril
lema = akrobat
lema = akrobatik
lema = akroléin
lema = akromat
lema = akronim
lema = akrual
lema = aksa
lema = aksara
lema = aksarawan
lema = aksarawati
lema = akséf
lema = aksélerasi
lema = akséleratif
lema = aksélerator
lema = aksén
lema = akséntuasi
lema = aksép
lema = akséptabel
lema = akséptan
lema = aks
sublema = éptasi
lema = akséptir
sublema = mengakséptir
lema = akséptor
lema = aksés
lema = aksési
lema = aksésibilitas
lema = aksésor
lema = aksi
lema = –
lema = aksial
lema = aksidéntal
sublema = aksidéntil
lema = aksidéntalia
lema = aksioma
lema = aksiomatis
lema = aksis
lema = Akt
lema = akta
lema = akte
lema = mengakte-notariskan
lema = aktentas
lema = aktif
sublema = ketidak-aktifan
lema = aktifi
sublema = s
lema = aktifi
sublema = tas
lema = akting
lema = aktiva
lema = aktivis
lema = aktor
lema

lema = angkuh
lema = angkuk
lema = angkul-angkul
lema = angkup
lema = angkup
lema = angkup-angkup
lema = angkur
lema = angkus(a)
lema = Angkuta
lema = angkut-angkut
lema = anglap
sublema = mengan
sublema = glap
sublema = nganglap
lema = angler
lema = angler
lema = Anglikan
lema = angling-anglingan
lema = Anglir
lema = Anglisisme
lema = anglo
lema = anglong
sublema = nganglong
lema = anglong
similar = anglung
lema = angmor
lema = angob
lema = Angola
lema = angomahi
lema = angon
lema = angop
lema = angot
lema = angpau
lema = angsa
lema = angsa
lema = angsana
lema = angsang
lema = angseg
similar = angsek
sublema = mengangseg
sublema = ngangseg
lema = angséna
lema = angsiau
similar = angsio
lema = angso
lema = angsok
sublema = a
lema = angsong
sublema = ngangsongin
lema = angsu
lema = angsung
lema = angsur
lema = anjing
lema = angur
lema = angus
lema = angut
lema = anhang
lema = anhu
lema = ani
lema = ani
lema = ani-ani
lema = aniaya
lema = ani(e)m
lema = anih
lema = anikterik
lema = anili

lema = apsari
lema = apsir
lema = apuah
lema = apu-apu
lema = apuh
lema = apukat
lema = apung
lema = apuran
lema = apus
lema = apus
sublema = ngapusi
lema = apus
lema = aq
lema = Aqasa
lema = aqidah
lema = arang
lema = aqil
lema = aql
lema = ‘Aqrab
similar = ‘Aqrob
lema = aquapolis
lema = ar
lema = ara
lema = ara
lema = Arab
lema = Arabica
lema = Arabisasi
lema = Arabusta
lema = A’raf
lema = Arafah
similar = Arafat
lema = Arafura
lema = arah
lema = arah
sublema = arah-arah
lema = mengarah-arah
lema = arah
lema = arai
lema = arai
lema = arak
lema = arak
lema = arak
lema = arakian
lema = aral
lema = aram
similar = pengaram
lema = aram
lema = arang
lema = arang
lema = arang-arang
lema = arang-arangan
lema = aran(g)semén
lema = aran(g)sé
similar = aran(g)sir
sublema = mengaran(g)sir
lema = pengaran(g)sir
lema = arap
lema = ngarapin
lema = ararut
lema = aras
lema = aras
lema = aras
lema = aras
lema = aras
sublema = aras-arasan
lema = aras
sublema = arash
similar = arasy
lema = arau
lema = a

lema = atol
lema = atoli
lema = atom
lema = Atomita
lema = Atoni
lema = Atonik
lema = atop
lema = atopik
lema = atos
lema = atraksi
lema = atraktif
lema = atraz
lema = atrét
lema = mengatrétkan
lema = atribusi
lema = atributif
sublema = atributip
lema = atrofi
lema = atsir
lema = atsiri
lema = atu
lema = Atu(d)
lema = atuh
lema = atuk
lema = atung
lema = mengatung
similar = teratung-atung
lema = atur
sublema = meng-atur-tepatkan
lema = pengatur-tepatan
lema = mengaturkan
lema = aturan
lema = pengatur
lema = atur
sublema = mengaturkan
lema = atur-atur
lema = atus
lema = AU
lema = aubade
lema = audién(si)
lema = audio
lema = audiograf
lema = audiogram
lema = audiologi
lema = audiométer
lema = audiométri
lema = audiovidéofi
sublema = l
lema = audiovisual
sublema = audiovisuil
lema = audisi
lema = audit
lema = mengaudit
lema = pengaudit
lema = peng
sublema = auditan
lema = auditorium
lema = audzubillah
sublema = a’udzubillah
similar = aud(z)ubillahi
lema = auk
lema = auksi
lema = aula
lema